<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Exps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "git+https://github.com/Karasiari/graphmcfexps.git"

  Cloning https://github.com/Karasiari/graphmcfexps.git to /tmp/pip-req-build-cs0rv47a
  Running command git clone --filter=blob:none --quiet https://github.com/Karasiari/graphmcfexps.git /tmp/pip-req-build-cs0rv47a
  Resolved https://github.com/Karasiari/graphmcfexps.git to commit 8a0ebe207142f17cd2ad16a33722956efceb7acd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for graphmcfexps: filename=graphmcfexps-0.1.0-py3-none-any.whl size=11507 sha256=5bb48be5811ed0fb560e84643bae864b9e0cfbac5a909ef6dc91e44d248e2baa
  Stored in directory: /tmp/pip-ephem-wheel-cache-7h7bs4yt/wheels/09/cf/d5/20f73fe3b91fddd32c1c90d1bb788e0d83851f63661cafc4ad
Successfully built graphmcfexps


In [2]:
import pandas as pd
import numpy as np
import os # Выберите, откуда тяните папку
from google.colab import files

BITRATE_DENOMINATOR = 100

# По папке формируем словарь графов со структурой -
#                                                {'название графа (как в папке)':
#                                                       {'adj_edges': список мультиребер смежности со значением capacity мультиребра,
#                                                        'traffic_edges': список мультребер запросов со значением weight мультиребра,
#                                                }

def csv_to_graph(path, demands_path, capacity_path):
  Df = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  Traffic = pd.read_csv(demands_path, header=None, names = ['id', 'source', 'target', 'bitrate'])
  Capacity = pd.read_csv(capacity_path, header=None, names = ['name', 'value'])
  df = Df.iloc[1:].copy()
  traffic = Traffic.iloc[1:].copy()
  capacity = Capacity.iloc[1:].copy()

  df['source'] = df['source'].astype(int)
  df['target'] = df['target'].astype(int)
  traffic['source'] = traffic['source'].astype(int)
  traffic['target'] = traffic['target'].astype(int)
  traffic['bitrate'] = traffic['bitrate'].astype(int) // BITRATE_DENOMINATOR
  capacity_value = (float(capacity[capacity['name']=='LineRate']['value']) // BITRATE_DENOMINATOR) * int(capacity[capacity['name']=='NumberOfWavelengths']['value'])

  sources = df['source'].tolist()
  targets = df['target'].tolist()
  sources_traffic = traffic['source'].tolist()
  targets_traffic = traffic['target'].tolist()
  bitrates_traffic = traffic['bitrate'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_edges = []
  traffic_edges = []

  for source, target in zip(sources, targets):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_edges.append((new_source, new_target, {"capacity": capacity_value}))
  for source, target, bitrate in zip(sources_traffic, targets_traffic, bitrates_traffic):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    traffic_edges.append((new_source, new_target, {"weight": bitrate}))

  return (adj_edges, traffic_edges)

def get_graphs(base_path, specified_graphs, specified):
  Graphs = {}
  csv_tables = []
  for folder_name in os.listdir(base_path):
    if folder_name in specified_graphs or not specified:
      folder_path = os.path.join(base_path, folder_name)
      csv_path = os.path.join(folder_path, 'links.csv')
      csv_path_demands = os.path.join(folder_path, 'demands.csv')
      csv_path_capacity = os.path.join(folder_path, 'params.csv')
      csv_tables.append(folder_name)

      (adj_edges, traffic_edges) = csv_to_graph(csv_path, csv_path_demands, csv_path_capacity)
      Graphs[folder_name] = {'adj_edges': adj_edges, 'traffic_edges': traffic_edges}

  return Graphs

path_to_folder = '/content/drive/MyDrive/Кола' # Свой путь

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import networkx as nx
import numpy as np

from graphmcfexps.expand_test import *

def hu_to_graphs(adj_edges, traffic_edges):
  adj_graph = nx.MultiGraph()
  adj_graph.add_edges_from(adj_edges)
  traffic_graph = nx.MultiDiGraph()
  traffic_graph.add_nodes_from(range(adj_graph.number_of_nodes()))
  traffic_graph.add_edges_from(traffic_edges)
  Graph = GraphMCFexps(adj_graph, traffic_graph)
  med_capacity = np.median([edge[2]["capacity"] for edge in adj_edges])

  return Graph, med_capacity

print(f'Доступные типы на распределение ресурсов: {get_available_types()}\n')
graphs_for_test_names = ['cola_t3', 'cola_t10', 'cola_t2', 'cola_t8', '2024_north', '2024_south', '2024_southwest']
graphs_for_test = get_graphs(path_to_folder, graphs_for_test_names, True)
for name, graph in graphs_for_test.items():
  adj_edges, traffic_edges = graph['adj_edges'], graph['traffic_edges']
  Graph, med_capacity = hu_to_graphs(adj_edges, traffic_edges)
  number_to_add = 4
  print(f'Добавим {number_to_add} новых мультиребер с capacity={med_capacity}\n')
  additional_capacities = [med_capacity] * number_to_add

  results = []
  for type in get_available_types():
    gamma = expand_network_test(additional_capacities, Graph, type, alpha_type="min_Lalpha") # здесь основная функция
    results.append((type, gamma))
  print(f'Для графа {name} результаты теста:\n{'\n'.join([str(type)+": "+str(gamma) for type, gamma in results])}\n')

Доступные типы на распределение ресурсов: ['initial', 'alpha', 'random', 'min_cut', 'min_Lalpha_cut', 'betweenness_unweighted']

Добавим 4 новых мультиребер с capacity=80.0

Для графа cola_t10 результаты теста:
initial: 1.9999999996333915
alpha: 1.9999999993560773
random: 1.9999999988443715
min_cut: 1.9999999983635477
min_Lalpha_cut: 1.9999999992812916
betweenness_unweighted: 1.9999999993519073

Добавим 4 новых мультиребер с capacity=80.0

Для графа cola_t3 результаты теста:
initial: 1.2698412697100192
alpha: 1.2820512819763026
random: 1.269841269816469
min_cut: 1.2698412696803374
min_Lalpha_cut: 1.2698412697690167
betweenness_unweighted: 1.2698412684229008

Добавим 4 новых мультиребер с capacity=80.0

Для графа cola_t8 результаты теста:
initial: 14.285714285295352
alpha: 15.384615382861192
random: 14.285714275188647
min_cut: 14.285714271894557
min_Lalpha_cut: 14.285714285119678
betweenness_unweighted: 14.28571426576951

Добавим 4 новых мультиребер с capacity=80.0

Для графа cola_t2 ре